In [1]:
import rdflib
import csv
from json import JSONDecodeError
from SPARQLWrapper import SPARQLWrapper, JSON

from rdflib import Graph
import pandas as pd

In [2]:
def GetClassLabel(URIString):
    if len(URIString.split(':')) > 2:
        Label = URIString.split(':')[-1]
    else:
        Label = URIString.split('/')[-1]
    return Label
    

In [3]:
mapping=pd.read_csv('../wd_class_name_mapping.csv', index_col=0)

In [9]:


ClassList = list(mapping['sch_id'])
sparq1 = SPARQLWrapper("https://yago-knowledge.org/sparql/query")

with open('YAGO_instances.csv', 'w') as f:
    writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    line = ['Class_Name', 'Class_URI', 'Instance_Name', 'Instance_URI']
    writer.writerow(line)
    for className in ClassList:
        MyList = []
        try:
            queryString = """
            PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
            PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
            PREFIX owl:<http://www.w3.org/2002/07/owl#>


            SELECT ?name
            WHERE {
             ?entity a  <http://schema.org/%s> .
             ?entity rdfs:label ?name.
              Filter (lang(?name)="en")
            }
                                        """ %(GetClassLabel(className))
            sparq1.setQuery(queryString)
            sparq1.setReturnFormat(JSON)
            results = sparq1.query().convert()

        except (TypeError, JSONDecodeError):
            continue

        if results == None:
            continue
        
        else:
            for r in results["results"]["bindings"]:
                MyList.append(r["name"]["value"])

            for i in range(len(MyList)):
                MyList[i] = MyList[i].replace(' ', '_')
                line = []
                line.append(GetClassLabel(className))
                line.append(className)
                line.append(GetClassLabel(MyList[i]))
                line.append(MyList[i])
                writer.writerow(line)

In [10]:
df=pd.read_csv('YAGO_instances.csv')
len(df.Class_Name.unique())
df['Instance_URI']=['https://yago-knowledge.org/resource/'+str(i) for i in df['Instance_URI']]
#df.to_csv('YAGO_Dataset.csv')